In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


data=pd.read_csv('Churn_Modelling.csv')
# data.head()
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()


label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])


ohe_geo=OneHotEncoder(sparse_output=False)
geo_encoded=ohe_geo.fit_transform(data[['Geography']])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df


## combine the encoded geography columns with the original data
data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)


## now split the data into features and targets
X = data.drop('EstimatedSalary',axis=1)
Y = data['EstimatedSalary']

## split the data in training and testing sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

## scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)



,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
X_train.shape

(8000, 12)

In [12]:
## ANN regression problem statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

## Build ANN model

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## 1ST hidden layer connected to input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid') ## output layer
])

## compile the model

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

# steup tensorboard
log_dir="regressionlogs/fit"
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

# setup earlystopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

#train the model
history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100, callbacks=[ tensorboard_callback,early_stopping_callback]
)


Epoch 1/100


C:\Users\Silajit Chakraborty\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 100430.3125 - mae: 100430.3125 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.2578 - mae: 100430.2578 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.3203 - mae: 100430.3203 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.3203 - mae: 100430.3203 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.2969 - mae: 100430.2969 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100430.2734 - mae: 100430.2734 - val_loss: 98725.0625 - val_mae: 98725.0625
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 24240), started 0:01:15 ago. (Use '!kill 24240' to kill it.)

In [26]:
## evaluate the model
test_loss,test_mae=model.evaluate(X_test,Y_test)
print(f"Test MAE : {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 98725.0625 - mae: 98725.0625    
Test MAE : 98725.0625


In [27]:
model.save('regressionmodel.h5')